## Fine tune a llama3.1 model with sagemaker

### Setup
Let's start by installing all the lib we need to do supervised fine-tuning. We're going to use

Transformers for the LLM which we're going to fine-tune
Datasets for loading a SFT dataset from the hub, and preparing it for the model
BitsandBytes and PEFT for fine-tuning the model on consumer hardware, leveraging Q-LoRa, a technique which drastically reduces the compute requirements for fine-tuning
TRL, a library which includes useful Trainer classes for LLM fine-tuning.

In [ ]:
#!pip install -q transformers[torch] datasets
#!pip install -q bitsandbytes trl peft
#!pip install flash-attn --no-build-isolation
#!pip install accelerate==0.29.3


In [3]:
import torch
torch.cuda.empty_cache()

In [4]:
import os

# Set path to config file
os.environ["SAGEMAKER_USER_CONFIG_OVERRIDE"] = os.getcwd()

### Load Data + Preprocessing
The dataset contains various splits, each with a certain number of rows. In our case, as we're going to do supervised fine-tuning (SFT), only the "train_sft" and "test_sft" splits are relevant for us.


In [5]:
from datasets import load_dataset, DatasetDict

INFO:datasets:PyTorch version 2.2.0 available.


In [6]:
# load data from s3 bucket
import boto3
import sagemaker

region_name = 'eu-central-1'

session = boto3.Session(region_name=region_name)
s3_sess = session.client('s3')
sm_session = sagemaker.Session(boto_session=session)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Fetched defaults config from location: /home/ec2-user/tmp/llm_fine_tuning/llm


In [7]:
training_input_path = f's3://{sm_session.default_bucket()}/datasets/writing_accuracy_dataset/train_dataset.json'
# define a data input dictonary with our uploaded s3 uris
data = {'training': training_input_path}

data

{'training': 's3://sagemaker-eu-central-1-505049265445/datasets/writing_accuracy_dataset/train_dataset.json'}

In [8]:
raw_dataset = load_dataset(
        "json",
        data_files=data
    )
raw_dataset

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


DatasetDict({
    training: Dataset({
        features: ['messages'],
        num_rows: 7528
    })
})

In [9]:
indices_1 = range(0,100)
indices_2 = range(101,201)
dataset_dict = {
    "train": raw_dataset["training"].select(indices_1),
    "test": raw_dataset["training"].select(indices_2)
}
raw_dataset = DatasetDict(dataset_dict)
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 100
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 100
    })
})

#### Tokenizer
Next, we instantiate the tokenizer, which is required to prepare the text for the model. The model doesn't directly take strings as input, but rather input_ids, which represent integer indices in the vocabulary of a Transformer model. 

We also set some attributes which the tokenizer of a base model typically doesn't have set, such as:

- the padding token ID. During pre-training, one doesn't need to pad since one just creates blocks of text to predict the next token, but during fine-tuning, we will need to pad the (instruction, completion) pairs in order to create batches of equal length.
- the model max length: this is required in order to truncate sequences which are too long for the model. Here we decide to train on at most 2048 tokens.
- the chat template. A chat template determines how each list of messages is turned into a tokenizable string, by adding special strings in between such as <|user|> to indicate a user message and <|assistant|> to indicate the chatbot's response. Here we define the default chat template, used by most chat models. See also the docs.

In [10]:
from transformers import AutoTokenizer
from huggingface_hub import login

In [11]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

# set pad_token_id equal to the eos_token_id if not set
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048

In [13]:
# use default template of instruct model
tokenizer.chat_template

'{{- bos_token }}\n{%- if custom_tools is defined %}\n    {%- set tools = custom_tools %}\n{%- endif %}\n{%- if not tools_in_user_message is defined %}\n    {%- set tools_in_user_message = true %}\n{%- endif %}\n{%- if not date_string is defined %}\n    {%- set date_string = "26 Jul 2024" %}\n{%- endif %}\n{%- if not tools is defined %}\n    {%- set tools = none %}\n{%- endif %}\n\n{#- This block extracts the system message, so we can slot it into the right place. #}\n{%- if messages[0][\'role\'] == \'system\' %}\n    {%- set system_message = messages[0][\'content\']|trim %}\n    {%- set messages = messages[1:] %}\n{%- else %}\n    {%- set system_message = "" %}\n{%- endif %}\n\n{#- System message + builtin tools #}\n{{- "<|start_header_id|>system<|end_header_id|>\\n\\n" }}\n{%- if builtin_tools is defined or tools is not none %}\n    {{- "Environment: ipython\\n" }}\n{%- endif %}\n{%- if builtin_tools is defined %}\n    {{- "Tools: " + builtin_tools | reject(\'equalto\', \'code_interp

#### Apply chat template
Once we have equipped the tokenizer with the appropriate attributes, it's time to apply the chat template to each list of messages. Here we basically turn each list of (instruction, completion) messages into a tokenizable string for the model.

Note that we specify tokenize=False here, since the SFTTrainer which we'll define later on will perform the tokenization internally. Here we only turn the list of messages into strings with the same format.

In [14]:
import re
import random
from multiprocessing import cpu_count

def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)

    return example

In [15]:
column_names = list(raw_dataset["train"].features)
column_names

['messages']

In [16]:
# applies the apply_chat_template function to each element in raw_dataset using multiple CPU cores, passing a tokenizer and removing specified columns, with a progress description.
raw_dataset = raw_dataset.map(apply_chat_template,
                                num_proc=cpu_count(),
                                fn_kwargs={"tokenizer": tokenizer},
                                remove_columns=column_names,
                                desc="Applying chat template")

In [17]:
train_data = raw_dataset["train"]
test_data = raw_dataset["test"]

#for index in random.sample(range(len(raw_dataset["train"])), 3):
  #print(f"Sample {index} of the processed training set:\n\n{raw_dataset['train'][index]['text']}")
  

In [18]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [19]:
import bitsandbytes as bnb

def find_all_linear_names(hf_model):
    lora_module_names = set()
    for name, module in hf_model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [20]:
from accelerate import Accelerator
from huggingface_hub import login
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sagemaker.remote_function import remote
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, set_seed
import transformers

# Start training
@remote(volume_size=50, job_name_prefix=f"train-{model_id.split('/')[-1].replace('.', '-')}")
def train_fn(
        model_name,
        train_ds,
        test_ds=None,
        lora_r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        num_train_epochs=1,
        gradient_checkpointing=False,
        merge_weights=False,
        seed=42,
        token=None
):
    set_seed(seed)

    accelerator = Accelerator()

    if token is not None:
        login(token=token)

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Set Tokenizer pad Token
    tokenizer.pad_token = tokenizer.eos_token

    with accelerator.main_process_first():
        # tokenize and chunk dataset
        lm_train_dataset = train_ds.map(
            lambda sample: tokenizer(sample["text"]), remove_columns=list(train_ds.features)
        )

        print(f"Total number of train samples: {len(lm_train_dataset)}")

        if test_ds is not None:
            lm_test_dataset = test_ds.map(
                lambda sample: tokenizer(sample["text"]), remove_columns=list(test_ds.features)
            )

            print(f"Total number of test samples: {len(lm_test_dataset)}")
        else:
            lm_test_dataset = None

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        quant_storage_dtype=torch.bfloat16
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        quantization_config=bnb_config,
        #attn_implementation="flash_attention_2",
        use_cache=False if gradient_checkpointing else True,
        cache_dir="/tmp/.cache"
    )

    model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=gradient_checkpointing)

    if gradient_checkpointing:
        model.gradient_checkpointing_enable()

    # get lora target modules
    modules = find_all_linear_names(model)
    print(f"Found {len(modules)} modules to quantize: {modules}")
    
    config = LoraConfig(
        r=lora_r,
        lora_alpha=lora_alpha,
        target_modules=modules,
        lora_dropout=lora_dropout,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)
    
    trainer = transformers.Trainer(
        model=model,
        train_dataset=lm_train_dataset,
        eval_dataset=lm_test_dataset if lm_test_dataset is not None else None,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            gradient_checkpointing=gradient_checkpointing,
            logging_strategy="steps",
            logging_steps=1,
            log_on_each_node=False,
            num_train_epochs=num_train_epochs,
            learning_rate=learning_rate,
            bf16=True,
            ddp_find_unused_parameters=False,
            save_strategy="no",
            output_dir="outputs"
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    )

    trainer.train()

    if merge_weights:
        output_dir = "/tmp/model"

        # merge adapter weights with base model and save
        # save int 4 model
        trainer.model.save_pretrained(output_dir, safe_serialization=False)
        # clear memory
        del model
        del trainer

        torch.cuda.empty_cache()

        # load PEFT model in fp16
        model = AutoPeftModelForCausalLM.from_pretrained(
            output_dir,
            low_cpu_mem_usage=True,
            torch_dtype=torch.float16,
            cache_dir="/tmp/.cache"
        )

        # Merge LoRA and base model and save
        model = model.merge_and_unload()
        model.save_pretrained(
            "/opt/ml/model", safe_serialization=True, max_shard_size="2GB"
        )
    else:
        trainer.model.save_pretrained("/opt/ml/model", safe_serialization=True)

    with accelerator.main_process_first():
        tokenizer.save_pretrained("/opt/ml/model")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.ImageUri
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.Dependencies
sagemaker.config INFO - Applied value from config key = SageMaker.PythonSDK.Modules.RemoteFunction.InstanceType


In [21]:
train_fn(
    model_id,
    train_ds=train_data,
    test_ds=test_data,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    num_train_epochs=1,
    merge_weights=True
)

2024-09-02 19:38:59,635 sagemaker.remote_function INFO     Serializing function code to s3://sagemaker-eu-central-1-505049265445/train-Meta-Llama-3-1-8B-Instruct-2024-09-02-19-38-59-635/function
2024-09-02 19:38:59,698 sagemaker.remote_function INFO     Serializing function arguments to s3://sagemaker-eu-central-1-505049265445/train-Meta-Llama-3-1-8B-Instruct-2024-09-02-19-38-59-635/arguments
2024-09-02 19:38:59,919 sagemaker.remote_function INFO     Copied dependencies file at './requirements.txt' to '/tmp/tmp982s5hom/temp_workspace/sagemaker_remote_function_workspace/requirements.txt'
2024-09-02 19:38:59,920 sagemaker.remote_function INFO     Successfully created workdir archive at '/tmp/tmp982s5hom/workspace.zip'
2024-09-02 19:38:59,945 sagemaker.remote_function INFO     Successfully uploaded workdir to 's3://sagemaker-eu-central-1-505049265445/train-Meta-Llama-3-1-8B-Instruct-2024-09-02-19-38-59-635/sm_rf_user_ws/workspace.zip'
2024-09-02 19:38:59,946 sagemaker.remote_function INFO

2024-09-02 19:39:01 Starting - Starting the training job
2024-09-02 19:39:01 Pending - Training job waiting for capacity...
2024-09-02 19:39:23 Pending - Preparing the instances for training...
2024-09-02 19:39:55 Downloading - Downloading input data...
2024-09-02 19:40:10 Downloading - Downloading the training image.